In [1]:
from fastapi import FastAPI
from pytrends.request import TrendReq
import pandas as pd
import time

# DOS OPCIONES, LA PRIMERA, CON LIMPIEZA COMO LLAMADA EXTERNA
app = FastAPI()

def obtener_datos_google_trends_semanal(topic):
    try:
        geo = ["lista_paises"] # Tenemos que incluirlos aun!! y revisar el resto de filtros, POR SI TENEMOS QUE METERLOS EN LA LLAMADA
        pytrends = TrendReq(hl='es', tz=360)
        pytrends.build_payload(kw_list=[topic], timeframe='2014-01-01 2024-03-31', geo=geo)
        data = pytrends.interest_over_time()
        data_semanal = data.resample('W').mean() # Convertir datos a frecuencia semanal, PERO NO FUNCIONAN, SALEN NAN, IMPUTAR!
        return data_semanal
    
    except Exception as e:
        print(f"Error al obtener datos de Google Trends: {e}") # Si ocurre un error, imprimirlo y esperar antes de volver a intentar, ES NECESARIO PARA EVITAR EL ERROR
        time.sleep(60)  # Esperar 60 segundos 
        return obtener_datos_google_trends_semanal(topic)

# ENDPOINT EXTRACCION
@app.get("/google-trends/{topic}") # donde meter los topics
def google_trends_handler(topic: str):
    trends_data = obtener_datos_google_trends_semanal(topic) # Obtenemos los datos de Google Trends para el tema proporcionado
    csv_data = trends_data.to_csv()
    return "200:ok" # Para confirmar que se ha descargado bien, O PODEMOS PONER CUALQUIER OTRA RESPUESTA

# FUNCION AUXILIAR LIMPIEZA (IGUAL QUE EN GDELT)
def limpiar_dataframe(df: pd.DataFrame):
    df_limpio= df.dropna().drop_duplicates()
    return df_limpio

@app.get("/limpieza/")
def endpoint_limpiar_dataframe(funcion_obtener_datos: str,topic: str):
    df = google_trends_handler(topic) # Para meter el topic
    df_limpio = limpiar_dataframe(df) # Llamar a la función para limpiar el DataFrame
    df_limpio.to_csv(f"csv_{funcion_obtener_datos}_limpios.csv")    # y lo guardamos limpio
     
    return {"mensaje": f"Datos de {funcion_obtener_datos} limpiados y guardados exitosamente"}


In [ ]:
# LA SEGUNDA, CON LA LIMPIEZA DENTRO DE LA LLAMADA INICIAL, sin endpoint de limpieza 
app = FastAPI()

def obtener_datos_google_trends_semanal(topic):
    try:
        geo = ["lista_paises"] # Tenemos que incluirlos aun!! y revisar el resto de filtros, POR SI TENEMOS QUE METERLOS EN LA LLAMADA
        pytrends = TrendReq(hl='es', tz=360)
        pytrends.build_payload(kw_list=[topic], timeframe='2014-01-01 2024-03-31', geo=geo)
        data = pytrends.interest_over_time()
        data_semanal = data.resample('W').mean() # Convertir datos a frecuencia semanal, 
        #PERO NO FUNCIONAN, SALEN NAN, IMPUTAR! O ELIMINAR ESTA FILA Y LOS DATOS SERAN MENSUALES
        return data_semanal
    
    except Exception as e:
        print(f"Error al obtener datos de Google Trends: {e}") # Si ocurre un error, imprimirlo y esperar antes de volver a intentar, ES NECESARIO PARA EVITAR EL ERROR
        time.sleep(60)  # Esperar 60 segundos 
        return obtener_datos_google_trends_semanal(topic)
    
# FUNCION AUXILIAR LIMPIEZA (IGUAL QUE EN GDELT)
def limpiar_dataframe(df: pd.DataFrame):
    df_limpio= df.dropna().drop_duplicates()
    return df_limpio

# ENDPOINT EXTRACCION Y LIMPIEZA
@app.get("/google-trends/{topic}") # donde meter los topics
def google_trends_handler(topic: str):
    trends_data = obtener_datos_google_trends_semanal(topic) # Obtenemos los datos de Google Trends para el tema proporcionado
    df_trend = pd.DataFrame(trends_data)
    df_limpiado = limpiar_dataframe(df_trend) # usando la funcion LIMPIAR DATAFRAME
    df_limpiado.to_csv("csv_trend_limpios.csv", index=False)
    csv_data = trends_data.to_csv() 
    return {"mensaje": "Datos de Trends limpiados y guardados exitosamente"} # Para confirmar que se ha descargado bien, O PODEMOS PONER CUALQUIER OTRA RESPUESTA
